<a href="https://colab.research.google.com/github/newstage75/python_data_analysis_100knock/blob/main/%E7%AC%AC3%E7%AB%A0/%E3%80%90%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%80%91%E9%A1%A7%E5%AE%A2%E3%81%AE%E5%85%A8%E4%BD%93%E5%83%8F%E3%82%92%E6%8A%8A%E6%8F%A1%E3%81%99%E3%82%8B10%E6%9C%AC%E3%83%8E%E3%83%83%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# 作業フォルダへの移動を行います。
# 人によって作業場所が異なるので、その場合作業場所を変更してください。
import os
os.chdir('/content/drive/MyDrive/データサイエンス/20240505python実践データ分析100本ノック/実習用Myフォルダ/第３章')

In [38]:
import pandas as pd
uselog = pd.read_csv('use_log.csv')
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [39]:
customer = pd.read_csv('customer_master.csv')
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [40]:
class_master = pd.read_csv('class_master.csv')
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,オールタイム,10500
1,C02,デイタイム,7500
2,C03,ナイト,6000


In [41]:
campaign_master = pd.read_csv('campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,通常
1,CA2,入会費半額
2,CA3,入会費無料


In [42]:
# 最初は先頭数行を表示させ、どのようなデータ列が存在するのか、それぞれのデータ列の関係性など、データの大枠を掴むことが重要。
# また、今回はデータの件数を把握するために、len()を用いてデータの表示も行なっている。

In [43]:
# legt_onとright_onが同じ変数名の場合は「on」で省略できる。（２行目にcampaign_idがその例。本当は１行目もonでいける。）
customer_join = pd.merge(customer, class_master, left_on='class', right_on='class', how='left')
customer_join = pd.merge(customer_join, campaign_master, on='campaign_id', how='left')
customer_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,ナイト,6000,通常


In [44]:
print(len(customer))
print(len(customer_join))

4192
4192


In [45]:
# 欠損値の確認（ジョインする際、キーが見つからないなど、上手くジョインができないと欠損値が自動で入る。）
# end_date（退会日）がないユーザーがあることが確認できた。
customer_join.isnull().sum()

,0
customer_id,0
name,0
class,0
gender,0
start_date,0
end_date,2842
campaign_id,0
is_deleted,0
class_name,0
price,0


# ノック23：顧客データの基礎集計をする。

In [46]:
customer_join.groupby('class_name').count()['customer_id']

,customer_id
class_name,
オールタイム,2045
デイタイム,1019
ナイト,1128


In [47]:
customer_join.groupby('campaign_name').count()['customer_id']


,customer_id
campaign_name,
入会費半額,650
入会費無料,492
通常,3050


In [48]:
customer_join.groupby('gender').count()['customer_id']

,customer_id
gender,
F,1983
M,2209


In [49]:
customer_join.groupby('is_deleted').count()['customer_id'] #end_dateの欠損2842と一致

,customer_id
is_deleted,
0,2842
1,1350


In [50]:
#キャンペーンコード(CA2：半額)の先頭をチェック
customer_join.loc[customer_join['campaign_id'] == 'CA2'].head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
840,IK123390,XXXXX,C01,F,2016-07-01 00:00:00,NaN,CA2,0,オールタイム,10500,入会費半額
841,HI880150,XXXXX,C03,F,2016-07-01 00:00:00,NaN,CA2,0,ナイト,6000,入会費半額
842,HD528660,XXXXX,C01,F,2016-07-01 00:00:00,NaN,CA2,0,オールタイム,10500,入会費半額
843,HD068817,XXXX,C03,F,2016-07-01 00:00:00,NaN,CA2,0,ナイト,6000,入会費半額
844,TS134180,XXX,C03,M,2016-07-01 00:00:00,NaN,CA2,0,ナイト,6000,入会費半額


In [51]:
#キャンペーンコード(CA2：半額)の末尾をチェック
customer_join.loc[customer_join['campaign_id'] == 'CA2'].tail()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
3345,HI680854,XXXX,C01,F,2018-06-14 00:00:00,2019-01-31 00:00:00,CA2,1,オールタイム,10500,入会費半額
3346,HD058348,XXXX,C01,M,2018-06-14 00:00:00,2018-11-30 00:00:00,CA2,1,オールタイム,10500,入会費半額
3347,HI312309,XXX,C01,M,2018-06-15 00:00:00,NaN,CA2,0,オールタイム,10500,入会費半額
3348,GD324324,XXX,C02,M,2018-06-15 00:00:00,2019-02-28 00:00:00,CA2,1,デイタイム,7500,入会費半額
3349,TS205006,XXXX,C01,F,2018-06-15 00:00:00,2019-01-31 00:00:00,CA2,1,オールタイム,10500,入会費半額


In [52]:
#キャンペーンコード(CA3：無料)の先頭をチェック
customer_join.loc[customer_join['campaign_id'] == 'CA3'].head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
2336,IK524314,XXXX,C01,M,2017-12-01 00:00:00,2018-05-31 00:00:00,CA3,1,オールタイム,10500,入会費無料
2337,TS218627,XXXXX,C02,F,2017-12-01 00:00:00,2018-07-31 00:00:00,CA3,1,デイタイム,7500,入会費無料
2338,GD401728,XXXXX,C01,M,2017-12-01 00:00:00,NaN,CA3,0,オールタイム,10500,入会費無料
2339,PL147335,XXXXXX,C03,M,2017-12-01 00:00:00,NaN,CA3,0,ナイト,6000,入会費無料
2340,IK142972,XXXXX,C01,F,2017-12-01 00:00:00,NaN,CA3,0,オールタイム,10500,入会費無料


In [53]:
#キャンペーンコード(CA3：無料)の末尾をチェック
customer_join.loc[customer_join['campaign_id'] == 'CA3'].tail()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
3958,PL405819,XXXX,C01,M,2018-12-15 00:00:00,NaN,CA3,0,オールタイム,10500,入会費無料
3959,HI958991,XXXXX,C01,F,2018-12-15 00:00:00,NaN,CA3,0,オールタイム,10500,入会費無料
3960,IK039344,XXX,C03,M,2018-12-15 00:00:00,NaN,CA3,0,ナイト,6000,入会費無料
3961,TS950243,XXXX,C01,M,2018-12-15 00:00:00,NaN,CA3,0,オールタイム,10500,入会費無料
3962,AS912483,XX,C03,M,2018-12-15 00:00:00,NaN,CA3,0,ナイト,6000,入会費無料


In [54]:
# customer_master.csvは2019年3月末時点での会員データとなる、ただし2018年度に退会した会員もデータとして存在する。
# ここで、2018年度に入会した人数を確認する。（1361名で、退会の1350人より11名多いことを確認）
customer_join['start_date'] = pd.to_datetime(customer_join['start_date'])
customer_start = customer_join.loc[customer_join['start_date'] > pd.to_datetime('20180401')]
print(len(customer_start))

1361


## ノック24：最新顧客の基礎集計

In [55]:
customer_join["end_date"] = pd.to_datetime(customer_join["end_date"])
# 条件をorにすることで、退会したユーザーのうち直近の2019年3月31日退会したユーザーは含むようにする。
customer_newer = customer_join.loc[(customer_join["end_date"] >= pd.to_datetime("20190331")) | (customer_join["end_date"].isna())]
print(len(customer_newer))
customer_newer["end_date"].unique()

2953


<DatetimeArray>
['NaT', '2019-03-31 00:00:00']
Length: 2, dtype: datetime64[ns]

In [56]:
# 会員区分
customer_newer.groupby('class_name').size()

,0
class_name,
オールタイム,1444
デイタイム,696
ナイト,813


In [57]:
# ちなみに書籍では、customer_idをカウントしていた。（同様の結果）
customer_newer.groupby('class_name').count()['customer_id']

,customer_id
class_name,
オールタイム,1444
デイタイム,696
ナイト,813


In [58]:
# キャンペーン区分
customer_newer.groupby('campaign_name').size()

,0
campaign_name,
入会費半額,311
入会費無料,242
通常,2400


In [59]:
# 性別毎
customer_newer.groupby('gender').size()

,0
gender,
F,1400
M,1553


## ノック25：利用履歴データを集計

In [60]:
uselog.head()

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [61]:
print(len(uselog))

197428


In [62]:
uselog["usedate"] = pd.to_datetime(uselog["usedate"])
uselog["年月"] = uselog["usedate"].dt.strftime("%Y%m")
uselog_months = uselog.groupby(["年月", "customer_id"], as_index=False).count()


In [63]:
uselog_months.head()

,年月,customer_id,log_id,usedate
0,201804,AS002855,4,4
1,201804,AS009013,2,2
2,201804,AS009373,3,3
3,201804,AS015315,6,6
4,201804,AS015739,7,7


In [64]:
uselog_months.rename(columns={"log_id":"count"}, inplace=True)
del uselog_months["usedate"]
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [66]:
# エラーが発生したため、count列が数値型か確認
print(uselog_months['count'].dtype)

int64


In [67]:
# uselog_customer = uselog_months.groupby("customer_id")
# print(uselog_customer['count'].dtype)

customer_id
AS002855    int64
AS008805    int64
AS009013    int64
AS009373    int64
AS015233    int64
            ...  
TS995853    int64
TS998593    int64
TS999079    int64
TS999231    int64
TS999855    int64
Name: count, Length: 4192, dtype: object


In [68]:
# uselog_customer.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36817,201903,TS958195,8
36818,201903,TS958303,9
36821,201903,TS960107,9
36835,201903,TS995238,7


In [71]:
# 顧客毎に絞り込み、月当たりの平均値、中央値、最大値、最小値を集計する。
# pandasのaggメソッドを使うことで、指定の列で集約している。（sum,mean,median,min,max,countなど）
# 数値型に対して、aggは指定しないといけないので、.["count"]は先に持ってくる！
# uselog_customer = uselog_months.groupby("customer_id").agg(["mean", "median", "max", "min"]).["count"]
uselog_customer = uselog_months.groupby("customer_id")["count"].agg(["mean", "median", "max", "min"])
uselog_customer = uselog_customer.reset_index(drop=False)
uselog_customer.head()

,customer_id,mean,median,max,min
0,AS002855,4.500000,5.0,7,2
1,AS008805,4.000000,4.0,8,1
2,AS009013,2.000000,2.0,2,2
3,AS009373,5.083333,5.0,7,3
4,AS015233,7.545455,7.0,11,4
